In [19]:
from math import lcm
from random import randrange

import numpy as np

from asymmetric import Asymmetric
from number import Number
from pure import is_prime
from restrictions.ring import Ring


class RSA(Asymmetric):

    def __init__(self, bits: int) -> None:
        self.n = self.generate_n(bits)
        self._public_key, self._private_key = self.generate_keys()

    def encrypt(self, message: str) -> list[int]:
        e, n = self.public_key
        cipher: list[int] = [pow(ord(char), e, n) for char in message]
        return cipher

    def decrypt(self, encrypted_message: list[int]) -> list[int]:
        d, n = self.private_key
        decrypted_numbers = [pow(char, d, n) for char in encrypted_message]
        return decrypted_numbers

    @staticmethod
    def encrypt_with_known_key(message: str, public_key: tuple[int, int]) -> int:
        cipher: int = pow(int(message), public_key[0], public_key[1])
        return cipher

    @staticmethod
    def decrypt_with_known_key(encrypted_message: list[int], private_key: tuple[int, int]) -> list[int]:
        decrypted_numbers: list[int] = [
            pow(char, private_key[0], private_key[1]) for char in encrypted_message
        ]
        return decrypted_numbers

    def generate_keys(self) -> tuple[tuple[int, int]]:

        self.phi: int = lcm(self.p - 1, self.q - 1)
        self.ring = Ring(self.phi)

        candidate: int = randrange(1, self.phi)
        e = Number(candidate, self.ring)
        g = e.gcd(self.phi)
        while g != 1:
            candidate = randrange(1, self.phi)
            e = Number(candidate, self.ring)
            g = e.gcd(self.phi)

        temp = self.ring.mult_inverse(e.value)
        d = Number(temp, self.ring)

        return ((e.value, self.n), (d.value, self.n))

    def generate_n(self, bits: int) -> int:
        self.p: int = self.generate_large_prime(int(bits / 2))
        self.q: int = self.generate_large_prime(int(bits / 2))
        return self.p * self.q

    def generate_large_prime(self, bits: int) -> int:
        p = randrange((2 ** (bits - 1)) + 1, (2**bits) - 1)
        while not is_prime(p):
            p = randrange((2 ** (bits - 1)) + 1, (2**bits) - 1)
        return p
    
    @property
    def private_key(self):
        return self._private_key
    
    @property
    def public_key(self):
        return self._public_key
    
    @staticmethod
    def load_from_file(content: str) -> tuple[int, int]:
        keys: list[str] = content.split()
        return (int(keys[0]), int(keys[1]))


if __name__ == "__main__":
    r = RSA(1024)
    xd = np.array(
        [[60, 16, 201, 179], [53, 107, 179, 22], [60, 70, 202, 47], [139, 10, 244, 15]]
    )
    encrypted_aes_key = [r.encrypt(str(byte)) for line in xd for byte in line]
    for x in encrypted_aes_key:
        print(x)
    
    print(chr(97))

    #decrypted_integers = [r.decrypt()]
    # aes_key = np.array(decrypted_integers, dtype=np.uint8).reshape(4, 4)

    # assert xd.all() == aes_key.all()
    


[88358643200669689853193978215381911538433914806888577407813931403288147458490759183116226870945937172963422051660243151484350202768073595858675358473014323561412102252365371141132883675117697385208754962930409108764384313066397314377154253606627102065866924721322718156649573050219315593258129438732790020, 123775564336035053118565624928715366252780137565422732532467920722084711864649963951410398803171353581570207127937351624276002371762219492645263705723837200424884803439536296565276159075910706842240778584387858335753832336661813600621793447520145727555873159008524463551066744973756509172047526341548727807922]
[98014399518795059728416686115780577540394845033064732987242602394354251824129161501820425951284403624813531147742593476520153965407587042023033099156891861427693012032986105517854880306929888337399327217597219416412196475347961594103277394760081884916448043933021115107761793191505371483387773219755730722567, 8835864320066968985319397821538191153843391480688857740781393140328814

In [ ]:
import unittest
